In [1]:
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer, LlamaTokenizerFast, LlamaForCausalLM
from datasets import load_dataset

2023-10-25 15:41:07.334030: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-25 15:41:07.367099: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-25 15:41:07.942536: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import pandas as pd
import json

# Read data from the JSON file into a list of dictionaries

with open("kbase_all.json", "r") as json_file:
    data = json.load(json_file)

In [3]:
#data['data']

In [4]:
df = pd.DataFrame(data['data'])
df.head()

,question,answer
0,What is the citation for KBase?,The citation for KBase is as follows:\n\nArkin...
1,Who funds the Genomic Sciences Program DOE Sys...,The Genomic Sciences Program DOE Systems Biolo...
2,What are the four Award Numbers associated wit...,The four Award Numbers associated with the fun...
3,When was the documentation last modified?,The documentation was last modified 2 years ago.
4,"What topics are covered in the ""Getting Starte...","The ""Getting Started"" section of the documenta..."


In [5]:
print(df['question'][0],df['answer'][0])

What is the citation for KBase? The citation for KBase is as follows:

Arkin AP, Cottingham RW, Henry CS, Harris NL, Stevens RL, Maslov S, et al. KBase: The United States Department of Energy Systems Biology Knowledgebase. Nature Biotechnology. 2018;36: 566. doi: 10.1038/nbt.4163


In [6]:
def formatting_func(example):
    text = f"### Question: {example['question']}\n ### Answer: {example['answer']}"
    return text

f = open('kbase_all.json') 
data = json.load(f) 
res = []
for d in data['data']:
    res.append({'text': formatting_func(d)})
d = {"data": res}
with open('kbase_formatted_data.json', 'w') as f:
    json.dump(d, f)
    

# dataset = load_dataset("json",data_files="kbase_formatted_data.json", split="train", field='data')
dataset = load_dataset("json",data_files="kbase_formatted_data.json", field='data')

dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 3040
    })
})

In [7]:
split_ds = dataset['train'].train_test_split(test_size=0.2)

In [8]:
split_ds['test']

Dataset({
    features: ['text'],
    num_rows: 608
})

In [9]:
def generate_prompt(data_point):
    return f"""
    <human>: {data_point['question']}
    <assistant>: {data_point['answer']}
    """.strip()

In [10]:
model_name = "llama-2-7b"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)
model_path = '/scratch/ac.pgupta/convLLM/llama2_model/llama-2-7b-chat-tok/' #"/scratch/ac.pgupta/finetune_llama/Nemo/llama2-7b-hf/"
tokenizer = LlamaTokenizerFast.from_pretrained(model_path)
model = LlamaForCausalLM.from_pretrained(model_path,
                                             device_map='auto',
                                             torch_dtype=torch.float16,quantization_config=bnb_config,
                                             )

# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     device_map = "auto",
#     quantization_config=bnb_config,
#     trust_remote_code=True
# )
model.config.use_cache = False

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [12]:
#tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [13]:
def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_prompt(data_point)
    tokenized_full_prompt = tokenizer(full_prompt,padding=True,truncation=True)
    return tokenized_full_prompt

In [15]:
# train_data = split_ds['train'].map(generate_and_tokenize_prompt)
# val_data = split_ds['test'].map(generate_and_tokenize_prompt)

In [16]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [17]:
from peft import LoraConfig, get_peft_model


lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
    "q_proj",
    "v_proj",
    ]
)
model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 33554432 || all params: 3533967360 || trainable%: 0.9494833591219133


In [18]:
from transformers import TrainingArguments

output_dir = "./results"
per_device_train_batch_size = 1
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 10
logging_steps = 10
learning_rate = 3e-5
max_grad_norm = 0.3
max_steps = 1000
warmup_ratio = 0.03
lr_scheduler_type = "cosine"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    #group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

In [20]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=split_ds['train'],
    peft_config=config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)
# trainer = transformers.Trainer(
#     model=model,
#     train_dataset = train_data,
#     eval_dataset = val_data,
#     data_collator = transformers.DataCollatorForLanguageModeling(tokenizer,mlm=False),
#     args=training_arguments,
# )

Map:   0%|          | 0/2432 [00:00<?, ? examples/s]

/scratch/ac.pgupta/finetune_Falcon7b/env/lib/python3.11/site-packages/trl/trainer/sft_trainer.py:207: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [21]:
model.config.use_cache = False

In [22]:
import wandb
wandb.init(project="finetune llama-sftt",
           config={
               "batch_size": None,
               "learning_rate": 3e-5,
               "dataset": "kbase_docs",
           })

wandb: Currently logged in as: prachigupta (prachi_nlp). Use `wandb login --relogin` to force relogin


In [ ]:
%%wandb
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/scratch/ac.pgupta/finetune_Falcon7b/env/lib/python3.11/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,2.353300
20,2.418900
30,2.233200
40,2.219900
50,2.122900
60,1.963300
70,1.925000
80,1.926500
90,1.858800
100,1.795300


/scratch/ac.pgupta/finetune_Falcon7b/env/lib/python3.11/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/scratch/ac.pgupta/finetune_Falcon7b/env/lib/python3.11/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/scratch/ac.pgupta/finetune_Fa

In [74]:
trainer.train()

wandb: Currently logged in as: prachigupta (prachi_nlp). Use `wandb login --relogin` to force relogin


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/scratch/ac.pgupta/finetune_Falcon7b/env/lib/python3.11/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,2.310200
20,2.104300
30,1.658800
40,1.584100
50,1.521300
60,1.374900
70,1.303300
80,1.243200
90,1.256400
100,1.202300


/scratch/ac.pgupta/finetune_Falcon7b/env/lib/python3.11/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/scratch/ac.pgupta/finetune_Falcon7b/env/lib/python3.11/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/scratch/ac.pgupta/finetune_Fa

TrainOutput(global_step=500, training_loss=1.2379826793670654, metrics={'train_runtime': 1612.8498, 'train_samples_per_second': 1.24, 'train_steps_per_second': 0.31, 'total_flos': 2.000497359584256e+16, 'train_loss': 1.2379826793670654, 'epoch': 0.82})

In [33]:
model.config.use_cache = True
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): PeftModelForCausalLM(
      (base_model): LoraModel(
        (model): RWForCausalLM(
          (transformer): RWModel(
            (word_embeddings): Embedding(65024, 4544)
            (h): ModuleList(
              (0-31): 32 x DecoderLayer(
                (input_layernorm): LayerNorm((4544,), eps=1e-05, elementwise_affine=True)
                (self_attention): Attention(
                  (maybe_rotary): RotaryEmbedding()
                  (query_key_value): Linear4bit(
                    in_features=4544, out_features=4672, bias=False
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=4544, out_features=64, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=64, out_featu

In [35]:
model.save_pretrained('trained_model')

In [43]:
from huggingface_hub import notebook_login
notebook_login()